In [1]:
# import libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation
import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import numpy as np
import math
from IPython.display import HTML
from matplotlib import rc
import pandas as pd
import numpy.lib.recfunctions as rf
from queue import Queue
import collections
import sys
import toml
from mesa.batchrunner import batch_run


This code runs with mesa version 3.00 or higher

In [2]:
# check mesa version
print("Mesa version: ", mesa.__version__)

Mesa version:  3.1.0


In [3]:
# Function to calculate Euclidean distance (distance between two points)
def euclidean_distance(pos1, pos2):
    # Use the formula to calculate distance between two coordinates
    return math.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)

# Function to calculate integer grid points between two coordinates connected in a straight line; returns a list of grid points
def connect2(ends):
    d0, d1 = np.diff(ends, axis=0)[0]
    if np.abs(d0) > np.abs(d1):
        return np.c_[np.arange(ends[0, 0], ends[1,0] + np.sign(d0), np.sign(d0), dtype=np.int32),
                     np.arange(ends[0, 1] * np.abs(d0) + np.abs(d0)//2,
                               ends[0, 1] * np.abs(d0) + np.abs(d0)//2 + (np.abs(d0)+1) * d1, d1, dtype=np.int32) // np.abs(d0)]
    else:
        return np.c_[np.arange(ends[0, 0] * np.abs(d1) + np.abs(d1)//2,
                               ends[0, 0] * np.abs(d1) + np.abs(d1)//2 + (np.abs(d1)+1) * d0, d0, dtype=np.int32) // np.abs(d1),
                     np.arange(ends[0, 1], ends[1,1] + np.sign(d1), np.sign(d1), dtype=np.int32)]

# Transforms the raw model grid data into usable arrays for the unblocked vision function
def prep_vision(vision_area, obstacles):
    # Find all obstacles within the vision range
    vision_array = np.asarray(vision_area)
    obstacles = np.asarray(obstacles)
    nrows, ncols = vision_array.shape
    dtype={'names':['f{}'.format(i) for i in range(ncols)], 'formats':ncols * [vision_array.dtype]}
    obstacles_in_sight = np.intersect1d(vision_array.view(dtype), obstacles.view(dtype))
    obstacles_in_sight = rf.structured_to_unstructured(obstacles_in_sight).tolist()

    # Trim vision_array down to only the edges (optimisation)
    xtrim = np.array([np.min(vision_array[:, 0]), np.max(vision_array[:, 0])])
    ytrim = np.array([np.min(vision_array[:, 1]), np.max(vision_array[:, 1])])

    xmask = np.isin(element = vision_array[:, 0], test_elements=xtrim)
    ymask = np.isin(element = vision_array[:, 1], test_elements=ytrim)

    xarray = vision_array[xmask]
    yarray = vision_array[ymask]
    vision_array = np.vstack((xarray, yarray))

    return vision_array, obstacles_in_sight

# Function to find all visible tiles to an agent around obstacles; call this function for enhanced vision behaviour
def unblocked_vision(pos, vision_area, obstacles):
    # Initiate data
    vision_array, obstacles_in_sight = prep_vision(vision_area, obstacles)
    x0 = pos[0]
    y0 = pos[1]
    vis_area = []

    # Iterate over all the tiles within the vision area
    for tile in vision_array[:, 0:2]:
        x1 = tile[0]
        y1 = tile[1]
        # Calculate simple sightline for straight lines (2D array); not covered by function connect2
        if x0 == x1:
            if y0 > y1:
                liny = np.vstack(np.arange(y1, y0 + 1)[::-1])
            else:
                liny = np.vstack(np.arange(y0, y1 + 1))
            linx = (np.ones((len(liny[:, 0]), 1))*x0).astype(int)
            visline = np.hstack((linx, liny))
        elif y0 == y1:
            if x0 > x1:
                linx = np.vstack(np.arange(x1, x0 + 1)[::-1])
            else:
                linx = np.vstack(np.arange(x0, x1 + 1))
            liny = (np.ones((len(linx[:, 0]), 1))*y0).astype(int)
            visline = np.hstack((linx, liny))

        # Use function to obtain sightline for diagonals (2D array)
        else:
            # Transform coordinates to required format for sightline function
            coords = np.array([[x0, y0],
                               [x1, y1]])
            visline = connect2(coords)

        # Add all possible tiles in sight (not blocked by an obstacle) to new vision list of tuples
        for loc in visline[:, 0:2]:
            vis_area.append((int(loc[0]), int(loc[1])))
            if [loc[0], loc[1]] in obstacles_in_sight:
                break

    # Remove duplicates from the vision list
    vision_area = list(set(vis_area))

    return vision_area, obstacles_in_sight

# Tool to plot the vision area of a single agent; only use for development and verification purposes
def plot_vision(pos, vision, obstacles, vision_area):
    viz = np.zeros((vision*2 + 1, vision*2 + 1))
    posx = pos[0] - vision
    posy = pos[1] - vision
    # add all different tiles to 2d spatial grid
    for i in range(len(viz[0, :])):
        for j in range(len(viz[:, 0])):
            y = vision*2 - j
            if (i + posx, j + posy) in vision_area:
                viz[y, i] = 1
            if (i + posx, j + posy) in obstacles:
                viz[y, i] = 2
            if (i + posx, j + posy) == pos:
                viz[y, i] = 3

    # plotting the grid
    fig, ax = plt.subplots(figsize=(4, 4))
    cmap = mcolors.ListedColormap(['black', 'green', 'white', 'red'])
    bounds = [0, 1, 2, 3]
    norm = mcolors.BoundaryNorm(bounds, cmap.N, extend='max')
    ax.imshow(viz, cmap=cmap, norm=norm)

# Function that generates the shortest path in grid tile coordinates to a given index point on the grid, avoiding obstacles
def exit_path(grid, pos, index):
    queue = collections.deque([[pos]])
    seen = set([pos])
    # Iterate over the path until the shortest path between pos and index locations is found
    while queue:
        path = queue.popleft()
        x, y = path[-1]
        if grid[y][x] == index:
            return path
        for x2, y2 in ((x+1,y), (x-1,y), (x,y+1), (x,y-1)):
            if 0 <= x2 < len(grid[:, 0]) and 0 <= y2 < len(grid[0, :]) and grid[y2][x2] != 1 and (x2, y2) not in seen:
                queue.append(path + [(x2, y2)])
                seen.add((x2, y2))

# Base Model

In [4]:
# import simulation settings
settings = toml.load("settings.toml")

###################
### AGENT CLASS ###
###################

# Define the StaffAgent class
class StaffAgent(Agent):
    def __init__(self, model, agent_vision): # Default vision range of 5 cells
        super().__init__(model)  # MESA `Agent` class initialization, auto-assigns unique_id in Mesa 3.0
        # Attributes of each agent
        self.found_exit = False  # Track if agent has reached the exit
        self.previous_pos = None  # Previous position of the agent
        self.real_vision = settings['run_params']['real_vision'] # Boolean; use vision blocked by obstacles
        self.vision = agent_vision  # Vision range of the agent
        self.floor = None # Stores the floor level agent is on
        self.path = None # stores current path to an exit
        self.changefloor = False # Boolean; indicates whether agent has changed between floor levels in previous step
        self.waiting = False # Track if the StaffAgent is waiting at the stairs on the 1st floor or not
        self.helping = False # Track if the StaffAgent is assigned to help some WheelchairAgent
        self.helping_agent = None # Unique ID of the wheelchair agent being helped by this agent

    # Function to move the agent towards a set of locations on the grid
    def move_towards(self, locations, maxpop=8, moore=True):
        self.previous_pos = self.pos  # Store the current position before moving
        # MESA `get_neighborhood` function retrieves nearby cells based on vision range
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=moore, include_center=False)

        min_distance = float('inf')  # Start with a very large distance
        best_step = None  # Initialize best step as None

        # Check each possible step to find the one closest to the exit
        for step in possible_steps:
            # Only consider steps that don't have obstacles and have less than 8 agents
            if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < maxpop:
                dist = []
                for loc in locations:
                    dist.append(euclidean_distance(step, loc))
                dist = np.min(dist)  # Distance to the exit
                if dist < min_distance:
                    min_distance = dist
                    best_step = step  # Update best step to be closer to the exit
        if best_step:
            # MESA `move_agent` function moves the agent to the new cell
            self.model.grid.move_agent(self, best_step)

    # Function to move the agent towards a desired exit by assigning them a path
    def move_to_exits(self):
        self.previous_pos = self.pos
        # If agent has a viable path, follow it
        if self.path and self.changefloor == False:
            self.path = self.path
            # If an agent is further along its path than it is expected to be, update the path
            if self.pos in self.path[1:]:
                self.path = self.path[self.path.index(self.pos):]
            # If an agent has strayed off its path, move back to the path
            elif self.pos != self.path[0]:
                path_grid = self.model.sgrid.copy()
                # store end goal of original path as index 11 on pathfinding grid and find new path
                path_grid[self.path[-1][1], self.path[-1][0]] = 11
                agent_path = exit_path(path_grid, self.pos, 11)
                set_agent_path = set(agent_path)
                # Find intersection points between paths and connect paths there to find the updated path
                intersect = next((a for a in self.path if a in set_agent_path), agent_path)
                path_grid[intersect[1], intersect[0]] = 10
                connect_path = exit_path(path_grid, self.pos, 10)[0:-1]
                self.path = connect_path + self.path[self.path.index(intersect):]
        # Else, call and store new path for the agent
        else:
            # Create new path for the staff agent to emergency exit (index 4)
            index = 4
            self.path = exit_path(self.model.sgrid, self.pos, index)
            self.changefloor = False
        # If there is a spot open in the desired path cell use that, else move to random cell
        if len(self.model.grid.get_cell_list_contents(self.path[1])) < settings['run_params']['tile_pop']:
            self.model.grid.move_agent(self, self.path[1])
            self.path = self.path[1:]
        else:
            possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            if self.floor == 0:
                valid_steps = [step for step in possible_steps if step not in self.model.obstacles and step not in self.model.stairs and len(self.model.grid.get_cell_list_contents(step)) < settings['run_params']['tile_pop']]
            else:
                valid_steps = [step for step in possible_steps if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < settings['run_params']['tile_pop']]
            if valid_steps:
                # Randomly choose a valid position and move there
                random_step = self.random.choice(valid_steps)
                # MESA `move_agent` function moves the agent to the chosen position
                self.model.grid.move_agent(self, random_step)
                # Update path with performed step
                self.path = [random_step] + self.path

    # Define the actions the agent will take in each step
    def step(self):
        # Assign a floor level to the agent
        if self.pos[1] > settings['map_params']['map_yhalf']:
            self.floor = 1
        if self.pos[1] <= settings['map_params']['map_yhalf']:
            self.floor = 0
        # If an agent is at the exit, mark as exited and remove from the floorplan
        if self.pos in self.model.exit_location:
            self.found_exit = True  # Set the agent's exit status to True
            self.model.grid.remove_agent(self)  # MESA function to remove the agent from the grid
            self.remove()  #self.remove() to remove from AgentSet
            self.model.cumulative_exited += 1  # Count this agent in cumulative exited agents
            self.model.exited_ids.append(self.unique_id)
        else:
            # MESA `get_neighborhood` checks the agent's vision area for the exit
            vision_area = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision, include_center=False)

            # If real vision (blocked by obstacles) is used, run vision functions
            if self.real_vision:
                vision_area, obstacles_in_sight = unblocked_vision(self.pos, vision_area, self.model.obstacles)

            # Tool for visualisation of vision area, uses plot_vision function
            # ONLY USE WITH 1 AGENT TO AVOID CRASHES
            # ------------------------------------ START -----------------------------------

            check_vis = False
            if check_vis:
                plot_vision(self.pos, self.vision, self.model.obstacles, vision_area)

            # ------------------------------------ END -----------------------------------
            # Find exits, stairs, and floor area within vision range
            exits = []
            for loc in self.model.exit_location:
                if loc in vision_area:
                    exits.append(loc)
            exit_in_vision = exits

            stairs = []
            for loc in self.model.stairs:
                if loc in vision_area:
                    stairs.append(loc)
            stairs_in_vision = stairs

            floor = []
            for loc in vision_area:
                if loc not in self.model.stairs and loc not in list(map(tuple, obstacles_in_sight)):
                    floor.append(loc)
            floor_in_vision = floor

            # Move towards the exit if it's in sight
            if exit_in_vision and not self.helping:
                self.move_towards(exit_in_vision, settings['run_params']['tile_pop'])
            # Move down the stairs if on the stairs
            elif self.pos in self.model.stairs and self.floor == 1:
                # print("staff:", self.unique_id)
                # print("staff helping attr:", self.helping)
                if not self.helping:
                    # print("staff going down")
                    self.model.grid.move_agent(self, (self.pos[0], self.pos[1]+settings['map_params']['stairs_jump']))
                    self.floor = 0
                    self.changefloor = True
                    self.waiting = False
                    if self.unique_id in self.model.waiting_staff:
                        self.model.waiting_staff.remove(self.unique_id)
                else:
                    self.waiting = True
                    if self.unique_id not in self.model.waiting_staff:
                        self.model.waiting_staff.append(self.unique_id)
            # Move towards stairs if in sight
            elif self.floor == 1 and stairs_in_vision and not self.helping:
                self.move_towards(stairs_in_vision, settings['run_params']['tile_pop'])
            # If on the stairs on the ground floor, move away from stairs
            elif self.pos in self.model.stairs and self.floor == 0 and not self.helping:
                self.move_towards(floor_in_vision, settings['run_params']['stair_pop'], moore=False)
            # Move towards wheelchair agent if close and in sight
            elif self.path and len(self.path) < 5 and self.helping and getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helping_agent)[0], "pos") in vision_area:
                pos = getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helping_agent)[0], "pos")
                self.move_towards([pos, pos], settings['run_params']['tile_pop'])
            # In case the helping agent strayed off course reroute its path
            elif self.helping and getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helping_agent)[0], "path")[-1] != self.path[-1]:
                print("Rerouted helping agent, path not the same")
                pathing_grid = self.model.sgrid.copy() # Initialise pathfinding grid
                # Mark the goal tile of the agent on the pathfinding grid and get the agent path towards it
                help_path = getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helping_agent)[0], "path")[-1]
                pathing_grid[help_path[-1][1], help_path[-1][0]] = 12
                self.path = exit_path(pathing_grid, self.pos, 12)
            # Move agent along its path
            elif not self.waiting:
                self.move_to_exits()


# Define the MobileAgent class
class MobileAgent(Agent):
    def __init__(self, model, agent_vision): # Default vision range of 5 cells
        super().__init__(model)  # MESA `Agent` class initialization, auto-assigns unique_id in Mesa 3.0
        # Attributes of each agent
        self.found_exit = False  # Track if agent has reached the exit
        self.previous_pos = None  # Previous position of the agent
        self.real_vision = settings['run_params']['real_vision'] # Boolean; use vision blocked by obstacles
        self.vision = agent_vision  # Vision range of the agent
        self.floor = None # Stores the floor level agent is on
        self.path = None # stores current path to an exit
        self.changefloor = False # Boolean; indicates whether agent has changed between floor levels in previous step
        self.random_n = np.random.random() # Gives a random value to the agent which is compared with settings
        self.following_staff = None # Unique id of the StaffAgent the agent is following
        self.staff_assigned = False # Boolean; Stores whether agent has been assigned to a staff in the previous step
        self.sign_vis = settings['run_params']['sign_vision'] # Fraction of agents who will react to a sign
        self.helping = False # Track if the MobileAgent is assigned to help some WheelchairAgent
        self.waiting = False# Track if the MobileAgent is waiting for the WheelchairAgent it is helping
        self.helping_agent = None # Unique ID of the wheelchair agent being helped by this agent

    # Function to move the agent towards a set of locations on the grid
    def move_towards(self, locations, maxpop=8, moore=True):
        self.previous_pos = self.pos  # Store the current position before moving
        # MESA `get_neighborhood` function retrieves nearby cells based on vision range
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=moore, include_center=False)

        min_distance = float('inf')  # Start with a very large distance
        best_step = None  # Initialize best step as None

        # Check each possible step to find the one closest to the exit
        for step in possible_steps:
            # Only consider steps that don't have obstacles and have less than 8 agents
            if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < maxpop:
                dist = []
                for loc in locations:
                    dist.append(euclidean_distance(step, loc))
                dist = np.min(dist)  # Distance to the exit
                if dist < min_distance:
                    min_distance = dist
                    best_step = step  # Update best step to be closer to the exit
        if best_step:
            # MESA `move_agent` function moves the agent to the new cell
            self.model.grid.move_agent(self, best_step)

    # Function to move the agent towards a desired exit by assigning them a path
    def move_to_exits(self):
        self.previous_pos = self.pos
        # If agent has a viable path, follow it
        if self.path and self.changefloor == False:
            self.path = self.path
            # If an agent is further along its path than it is expected to be, update the path
            if self.pos in self.path[1:]:
                self.path = self.path[self.path.index(self.pos):]
            # If an agent has strayed off its path, move back to the path
            elif self.pos != self.path[0]:
                path_grid = self.model.sgrid.copy()
                # store end goal of original path as index 11 on pathfinding grid and find new path
                path_grid[self.path[-1][1], self.path[-1][0]] = 11
                agent_path = exit_path(path_grid, self.pos, 11)
                set_agent_path = set(agent_path)
                # Find intersection points between paths and connect paths there to find the updated path
                intersect = next((a for a in self.path if a in set_agent_path), agent_path)
                path_grid[intersect[1], intersect[0]] = 10
                connect_path = exit_path(path_grid, self.pos, 10)[0:-1]
                self.path = connect_path + self.path[self.path.index(intersect):]
        # Else, call and store new path for the agent
        else:
            # Decide whether to go for emergency or main exit
            if self.random_n <= self.sign_vis or self.following_staff:
                # Emergency exit indicator for path finding algorithm
                index = 4
            else:
                # Main entrance indicator for path finding algorithm
                index = 3
            self.path = exit_path(self.model.sgrid, self.pos, index)
            self.changefloor = False
        # If there is a spot open in the desired path cell use that, else move to random cell
        if len(self.model.grid.get_cell_list_contents(self.path[1])) < settings['run_params']['tile_pop']:
            self.model.grid.move_agent(self, self.path[1])
            self.path = self.path[1:]
        else:
            possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            if self.floor == 0:
                valid_steps = [step for step in possible_steps if step not in self.model.obstacles and step not in self.model.stairs and len(self.model.grid.get_cell_list_contents(step)) < settings['run_params']['tile_pop']]
            else:
                valid_steps = [step for step in possible_steps if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < settings['run_params']['tile_pop']]
            if valid_steps:
                # Randomly choose a valid position and move there
                random_step = self.random.choice(valid_steps)
                # MESA `move_agent` function moves the agent to the chosen position
                self.model.grid.move_agent(self, random_step)
                # Update path with performed step
                self.path = [random_step] + self.path

    # Define the actions the agent will take in each step
    def step(self):
        # Assign a floor level to the agent
        if self.pos[1] > settings['map_params']['map_yhalf']:
            self.floor = 1
        if self.pos[1] <= settings['map_params']['map_yhalf']:
            self.floor = 0
        # If an agent is at the exit, mark as exited and remove from the floorplan
        if self.pos in self.model.exit_location:
            self.found_exit = True  # Set the agent's exit status to True
            self.model.grid.remove_agent(self)  # MESA function to remove the agent from the grid
            self.remove()  #self.remove() to remove from AgentSet
            self.model.cumulative_exited += 1  # Count this agent in cumulative exited agents
            self.model.exited_ids.append(self.unique_id)
        else:
            # MESA `get_neighborhood` checks the agent's vision area for the exit
            vision_area = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision, include_center=False)

            # If real vision (blocked by obstacles) is used, run vision functions
            if self.real_vision:
                vision_area, obstacles_in_sight = unblocked_vision(self.pos, vision_area, self.model.obstacles)

            # Tool for visualisation of vision area, uses plot_vision function
            # ONLY USE WITH 1 AGENT AND 1 TIMESTEP
            # ------------------------------------ START -----------------------------------

            check_vis = False
            if check_vis:
                plot_vision(self.pos, self.vision, self.model.obstacles, vision_area)

            # ------------------------------------ END -----------------------------------
            # Find exits, staff, stairs, and floor within vision range
            exits = []
            for loc in self.model.exit_location:
                if loc in vision_area:
                    exits.append(loc)
            exit_in_vision = exits

            staff_dict = {}
            for staff_unique_id in self.model.staff_locations:
                if self.model.staff_locations[staff_unique_id] in vision_area:
                    staff_dict[staff_unique_id] = self.model.staff_locations[staff_unique_id]
            staff_in_vision = staff_dict

            stairs = []
            for loc in self.model.stairs:
                if loc in vision_area:
                    stairs.append(loc)
            stairs_in_vision = stairs

            floor = []
            for loc in vision_area:
                if loc not in self.model.stairs and loc not in list(map(tuple, obstacles_in_sight)):
                    floor.append(loc)
            floor_in_vision = floor

            # Move towards the exit if it's in sight
            if exit_in_vision and not self.helping:
                self.move_towards(exit_in_vision, settings['run_params']['tile_pop'])
            # Move down the stairs if on the stairs
            elif self.pos in self.model.stairs and self.floor == 1:
                if not self.helping:
                    self.model.grid.move_agent(self, (self.pos[0], self.pos[1]+settings['map_params']['stairs_jump']))
                    self.floor = 0
                    self.changefloor = True
                    self.following_staff = None
                    self.waiting = False
                else:
                    self.waiting = True
            # Move towards stairs if in sight
            elif self.floor == 1 and stairs_in_vision and not self.helping:
                self.move_towards(stairs_in_vision, settings['run_params']['tile_pop'])
            # If on the stairs on the ground floor, move away from stairs
            elif self.pos in self.model.stairs and self.floor == 0 and not self.helping:
                self.move_towards(floor_in_vision, settings['run_params']['stair_pop'], moore=False)
            # Get the path of a staff if assigned to one and move along the path
            elif self.following_staff and not self.helping:
                # Crate a new path from the path of a staff if that has not already happened
                if self.following_staff in self.model.staff_locations and self.staff_assigned == True:
                    # Get the path of the staff agent and put the goal tile in the pathfinding grid
                    staff_path = getattr(self.model.agents.select(lambda agent: agent.unique_id == self.following_staff)[0], "path")
                    agent_path = None
                    # If statement to prevent crashes when staff do not yet have a path
                    if staff_path:
                        staff_grid = self.model.sgrid.copy()
                        staff_grid[staff_path[-1][1], staff_path[-1][0]] = 11

                        # Get the path of the agent to the staff path goal tile and find intersection between the two
                        agent_path = exit_path(staff_grid, self.pos, 11)
                    # If statement to prevent crashes when staff is on stairs
                    if agent_path:
                        set_agent_path = set(agent_path)
                        intersect = next((a for a in staff_path if a in set_agent_path), agent_path)
                        # Set the index of the tile where paths intersect to 10
                        staff_grid[intersect[1], intersect[0]] = 10

                        # find path to the intersection point and connect to truncated staff path
                        connect_path = exit_path(staff_grid, self.pos, 10)[0:-1]
                        self.path = connect_path + staff_path[staff_path.index(intersect):]
                        self.staff_assigned = False
                    # Move along the stored path
                self.move_to_exits()
            # Get tagged to a staff if it's not helping & staff in sight & it's not waiting and move towards it
            elif not self.helping and staff_in_vision and next(iter(staff_in_vision)) not in self.model.waiting_staff:
                self.following_staff = next(iter(staff_in_vision)) # unique id of the first staff in staff_in_vision
                staff_path = getattr(self.model.agents.select(lambda agent: agent.unique_id == self.following_staff)[0], "path")
                # Only get tagged to staff if its path is longer than 2 to prevent crashes with future pathfinding
                if staff_path and len(staff_path) > 2:
                    self.staff_assigned = True
                    self.move_towards([self.model.staff_locations[self.following_staff]], settings['run_params']['tile_pop'])
                else:
                    # If the staff is right next to an exit, agents don't get tagged to them and move along their own path
                    self.move_to_exits()
                    self.following_staff = None
            # Move towards wheelchair agent if close and in sight
            elif self.path and len(self.path) < 5 and self.helping and getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helping_agent)[0], "pos") in vision_area:
                pos = getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helping_agent)[0], "pos")
                self.move_towards([pos, pos], settings['run_params']['tile_pop'])
            # In case the helping agent strayed off course reroute its path
            elif self.helping and getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helping_agent)[0], "path")[-1] != self.path[-1]:
                print("Rerouted helping agent, path not the same")
                pathing_grid = self.model.sgrid.copy() # Initialise pathfinding grid
                # Mark the goal tile of the agent on the pathfinding grid and get the agent path towards it
                help_path = getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helping_agent)[0], "path")[-1]
                pathing_grid[help_path[-1][1], help_path[-1][0]] = 12
                self.path = exit_path(pathing_grid, self.pos, 12)
            # Move agent along its path
            elif not self.waiting:
                self.move_to_exits()

class WheelchairAgent(Agent):
    def __init__(self, model, agent_vision): # Default vision range of 5 cells
        super().__init__(model)  # MESA `Agent` class initialization, auto-assigns unique_id in Mesa 3.0
        # Attributes of each agent
        self.found_exit = False  # Track if agent has reached the exit
        self.previous_pos = None  # Previous position of the agent
        self.real_vision = settings['run_params']['real_vision'] # Boolean; use vision blocked by obstacles
        self.vision = agent_vision  # Vision range of the agent
        self.floor = None # Stores the floor level agent is on
        self.path = None # stores current path to an exit
        self.changefloor = False
        self.random_n = np.random.random() # Gives a random value to the agent which is compared with sign_vis
        self.following_staff = None #unique id of the StaffAgent it is following
        self.staff_assigned = False # Boolean; Stores whether agent has been assigned to a staff in the previous step
        self.sign_vis = settings['run_params']['sign_vision'] # Fraction of agents who will react to a sign
        self.being_helped = False # Tracks whether some agents ARE OR WERE assigned to help this WheelchairAgent
        self.helper_ids = [] # unique ids of helpers

    # Function to move the agent towards a set of locations on the grid
    def move_towards(self, locations, maxpop=8, moore=True):
        self.previous_pos = self.pos  # Store the current position before moving
        # MESA `get_neighborhood` function retrieves nearby cells based on vision range
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=moore, include_center=False)

        min_distance = float('inf')  # Start with a very large distance
        best_step = None  # Initialize best step as None

        # Check each possible step to find the one closest to the exit
        for step in possible_steps:
            # Only consider steps that don't have obstacles and have less than 8 agents
            if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < maxpop:
                dist = []
                for loc in locations:
                    dist.append(euclidean_distance(step, loc))
                dist = np.min(dist)  # Distance to the exit
                if dist < min_distance:
                    min_distance = dist
                    best_step = step  # Update best step to be closer to the exit
        if best_step:
            # MESA `move_agent` function moves the agent to the new cell
            self.model.grid.move_agent(self, best_step)

    # Function to move the agent towards a desired exit based on memory or signs
    def move_to_exits(self):
        self.previous_pos = self.pos
        # If agent has a viable path, follow it
        if self.path and self.changefloor == False:
            self.path = self.path
            # If an agent is further along its path than it is expected to be, update the path
            if self.pos in self.path[1:]:
                self.path = self.path[self.path.index(self.pos):]
            # If an agent has strayed off its path, move back to the path
            elif self.pos != self.path[0]:
                path_grid = self.model.sgrid.copy()
                # store end goal of original path as index 11 on pathfinding grid and find new path
                path_grid[self.path[-1][1], self.path[-1][0]] = 11
                agent_path = exit_path(path_grid, self.pos, 11)
                set_agent_path = set(agent_path)
                # Find intersection points between paths and connect paths there to find the updated path
                intersect = next((a for a in self.path if a in set_agent_path), agent_path)
                path_grid[intersect[1], intersect[0]] = 10
                connect_path = exit_path(path_grid, self.pos, 10)[0:-1]
                self.path = connect_path + self.path[self.path.index(intersect):]
        # Else, call and store new path for the agent
        else:
            # Decide whether to go for emergency exit or main exit
            if self.random_n <= self.sign_vis or self.following_staff and self.floor == 0:
                # Emergency exit indicator for path finding algorithm
                index = 4
            else:
                # Main entrance indicator for path finding algorithm
                index = 3
            self.path = exit_path(self.model.sgrid, self.pos, index)
            self.changefloor = False
        # If there is a spot open in the desired path cell use that, else move to random cell
        if len(self.model.grid.get_cell_list_contents(self.path[1])) < settings['run_params']['tile_pop']:
            self.model.grid.move_agent(self, self.path[1])
            self.path = self.path[1:]
        else:
            possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            if self.floor == 0:
                valid_steps = [step for step in possible_steps if step not in self.model.obstacles and step not in self.model.stairs and len(self.model.grid.get_cell_list_contents(step)) < settings['run_params']['tile_pop']]
            else:
                valid_steps = [step for step in possible_steps if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < settings['run_params']['tile_pop']]
            if valid_steps:
                # Randomly choose a valid position and move there
                random_step = self.random.choice(valid_steps)
                # MESA `move_agent` function moves the agent to the chosen position
                self.model.grid.move_agent(self, random_step)
                # Update path with performed step
                self.path = [random_step] + self.path

    # Define the actions the agent will take in each step
    def step(self):
        # Assign a floor level to the agent
        if self.pos[1] > settings['map_params']['map_yhalf']:
            self.floor = 1
        if self.pos[1] <= settings['map_params']['map_yhalf']:
            self.floor = 0
        # If an agent is at the exit, mark as exited and remove from the floorplan
        if self.pos in self.model.exit_location:
            self.found_exit = True  # Set the agent's exit status to True
            self.model.grid.remove_agent(self)  # MESA function to remove the agent from the grid
            self.remove()  #self.remove() to remove from AgentSet
            self.model.cumulative_exited += 1  # Count this agent in cumulative exited agents
            # self.model.cumulative_chair_agents_not_exited -= 1 # Dis-count this agent in cumulative WheelchairAgents yet to exit
        else:
            # MESA `get_neighborhood` checks the agent's vision area for the exit
            vision_area = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision, include_center=False)

            # If real vision (blocked by obstacles) is used, run vision functions
            if self.real_vision:
                vision_area, obstacles_in_sight = unblocked_vision(self.pos, vision_area, self.model.obstacles)

            # Tool for visualisation of vision area, uses plot_vision function
            # ONLY USE WITH 1 AGENT AND 1 TIMESTEP
            # ------------------------------------ START -----------------------------------

            check_vis = False
            if check_vis:
                plot_vision(self.pos, self.vision, self.model.obstacles, vision_area)

            # ------------------------------------ END -----------------------------------

            # Set path if on the first floor for other agents to interact with in the future
            if self.floor == 1 and not self.being_helped:
                self.move_to_exits()

            # Find nearest agents for help and assign to the wheelchair agent
            if self.floor == 1 and not self.being_helped:
                helping_agents = []
                # Find suitable (closest) agents from all agents active in the model until enough helpers are assigned
                for agent in self.model.agents:
                    if (agent in self.model.agents_by_type[StaffAgent] or agent in self.model.agents_by_type[MobileAgent]):
                        if not agent.helping and agent.floor == 1 and euclidean_distance(self.pos, agent.pos) <= settings['run_params']['neigh_dist']:
                            # Find distance between agents
                            dist = euclidean_distance(self.pos, agent.pos)
                            helping_agents.append((agent, dist))

                # Find the closest agent from the agents around
                if len(helping_agents) >= 2:
                    minlist = sorted(helping_agents, key=lambda agent: agent[1])
                    # Setting the path of the chosen helping agents (shortest distance to wheelchair agent)
                    agent_objs = [minlist[0][0], minlist[1][0]]
                    for agent in agent_objs:
                        wheelchair_grid = self.model.sgrid.copy() # Initialise pathfinding grid
                        # Mark the goal tile of the wheelchair agent on the pathfinding grid and get the agent path towards it
                        wheelchair_grid[self.path[-1][1], self.path[-1][0]] = 12
                        agent_path = exit_path(wheelchair_grid, agent.pos, 12)
                        # Set the path attribute of the helper agent
                        setattr(self.model.agents.select(lambda agent_l: agent_l.unique_id == agent.unique_id)[0], "path", agent_path)

                        # Set helping attribute of helping agents and pass ID of the wheelchair agent
                        setattr(self.model.agents.select(lambda agent_l: agent_l.unique_id == agent.unique_id)[0], "helping", True)
                        setattr(self.model.agents.select(lambda agent_l: agent_l.unique_id == agent.unique_id)[0], "helping_agent", self.unique_id)
                        self.helper_ids.append(agent.unique_id) # Store unique ID of helping agent

                    self.being_helped = True
                else:
                    print("Could not find someone to help")

            # Find exits, staff, stairs, and floor area within vision range
            exits = []
            for loc in self.model.exit_location:
                if loc in vision_area:
                    exits.append(loc)
            exit_in_vision = exits

            staff_dict = {}
            for staff_unique_id in self.model.staff_locations:
                if self.model.staff_locations[staff_unique_id] in vision_area:
                    staff_dict[staff_unique_id] = self.model.staff_locations[staff_unique_id]
            staff_in_vision = staff_dict

            floor = []
            for loc in vision_area:
                if loc not in self.model.stairs and loc not in list(map(tuple, obstacles_in_sight)):
                    floor.append(loc)
            floor_in_vision = floor

            # Don't move on first floor if not helped yet
            if self.being_helped or self.floor == 0:
                # Move towards the exit if it's in sight
                if exit_in_vision:
                    self.move_towards(exit_in_vision, settings['run_params']['tile_pop'])
                # Move down the stairs if on the stairs and helpers are with the wheelchair agent
                elif self.pos in self.model.stairs and self.floor == 1:
                    # if both the helpers are also on the same location or in the cells around it
                    helpcells = self.model.grid.get_neighborhood(self.pos, moore=True, radius=2, include_center=True)
                    if getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helper_ids[0])[0], "pos") in helpcells and getattr(self.model.agents.select(lambda agent: agent.unique_id == self.helper_ids[1])[0], "pos") in helpcells:
                        self.model.grid.move_agent(self, (self.pos[0], self.pos[1]+settings['map_params']['stairs_jump']))
                        self.floor = 0
                        self.changefloor = True
                        # set helping attributes of helping agents to False
                        setattr(self.model.agents.select(lambda agent: agent.unique_id == self.helper_ids[0])[0], "helping", False)
                        setattr(self.model.agents.select(lambda agent: agent.unique_id == self.helper_ids[1])[0], "helping", False)
                elif self.following_staff:
                    if self.following_staff in self.model.staff_locations and self.staff_assigned == True:
                        # Get the path of the staff agent and put goal tile in the pathfinding grid
                        staff_path = getattr(self.model.agents.select(lambda agent: agent.unique_id == self.following_staff)[0], "path")
                        agent_path = None
                        if staff_path:
                            staff_grid = self.model.sgrid.copy()
                            staff_grid[staff_path[-1][1], staff_path[-1][0]] = 11

                            # Get the path of the agent to the staff goal and find intersection between the two
                            agent_path = exit_path(staff_grid, self.pos, 11)
                        # If statement to prevent crashes when staff is on stairs
                        if agent_path:
                            set_agent_path = set(agent_path)
                            intersect = next((a for a in staff_path if a in set_agent_path), agent_path)
                            staff_grid[intersect[1], intersect[0]] = 10

                            # find path to the intersection point and connect to truncated staff path
                            connect_path = exit_path(staff_grid, self.pos, 10)[0:-1]
                            self.path = connect_path + staff_path[staff_path.index(intersect):]
                            self.staff_assigned = False
                    self.move_to_exits()
                # If on the stairs on the ground floor, move away from stairs
                elif self.pos in self.model.stairs and self.floor == 0:
                    self.move_towards(floor_in_vision, settings['run_params']['stair_pop'], moore=False)
                # Get tagged to a staff if staff in sight and move towards it
                elif staff_in_vision and self.floor == 0:
                    self.following_staff = next(iter(staff_in_vision)) # unique id of the first staff in staff_in_vision
                    staff_path = getattr(self.model.agents.select(lambda agent: agent.unique_id == self.following_staff)[0], "path")
                    if staff_path and len(staff_path) > 2:
                        self.staff_assigned = True
                        self.move_towards([self.model.staff_locations[self.following_staff]])
                    else:
                        self.move_to_exits()
                        self.following_staff = None
                else:
                    self.move_to_exits()

###################
### MODEL CLASS ###
###################

# Define the model class to handle the overall environment
class FloorPlanModel(Model):
    def __init__(self, num_staff, num_mobile_agents, num_chair_agents, tot_agents, agent_vision):
        super().__init__()  # `Model` class initialization

        # Basic model settings
        self.num_staff = num_staff
        self.num_mobile_agents = num_mobile_agents
        self.num_chair_agents = num_chair_agents
        self.agent_vision = agent_vision
        self.total_agents = tot_agents
        self.grid = MultiGrid(settings['map_params']['grid_x'], settings['map_params']['grid_y'], False)  # MESA grid with dimensions; False means no wrapping
        self.static_grid = None

        self.step_count = 0

        # -----------------------------------------------------------------------------------------------------------------------------
        # Initiate the floorplan and mutate the csv to fit the required array
        # -----------------------------------------------------------------------------------------------------------------------------
        floor0 = pd.read_csv(settings['run_params']['floorplan'], header=None)
        floor0 = pd.DataFrame.to_numpy(floor0)
        floor0 = np.flipud(floor0)

        # Make lists of all the different tile types
        GreenTiles = []
        BlackTiles = []
        WhiteTiles = []
        BlueTiles = []
        PurpleTiles = []
        YellowTiles = []
        OrangeTiles = []
        RedTiles = []

        for i in range(np.shape(floor0)[0]):
            for j in range(np.shape(floor0)[1]):
                if floor0[i, j] == 0:
                    GreenTiles.append((j, i))
                if floor0[i, j] == 1:
                    BlackTiles.append((j, i))
                if floor0[i, j] == 2:
                    WhiteTiles.append((j, i))
                if floor0[i, j] == 3:
                    BlueTiles.append((j, i))
                if floor0[i, j] == 4:
                    PurpleTiles.append((j, i))
                if floor0[i, j] == 5:
                    YellowTiles.append((j, i))
                if floor0[i, j] == 6:
                    OrangeTiles.append((j, i))
                if floor0[i, j] == 7:
                    RedTiles.append((j, i))

        # Save tile type lists to class objects
        self.outdoors = GreenTiles
        self.obstacles = BlackTiles
        self.indoors = WhiteTiles + BlueTiles # Walking area + doors
        self.exit_location = PurpleTiles
        self.stairs = YellowTiles
        self.lifts = OrangeTiles

        self.staff_locations = {} # initiate an empty dictionary to store locations of StaffAgents
        self.waiting_staff = [] # initiate an empty list to store unique ids of waiting StaffAgents

        # Adapt grid for path finding algorithm
        # Outside + obstacles = 1, Walking space = 2, entrance finding = 3, emergency exit finding = 4, lift finding = 7
        floor0[floor0 == 0] = 1
        floor0[(floor0 == 3) | (floor0 == 7)] = 2
        # Set stairs on top floor as emergency exits to navigate towards and set lifts on top floor as their own index
        topfloor = floor0[63:127]
        topfloor[topfloor == 5] = 4
        topfloor[topfloor == 6] = 7
        # Recombine floors and set main stairs + main exit as main index (3)
        floor0[63:127] = topfloor
        floor0[30, 125] = 3
        floor0[90, 44] = 3
        self.sgrid = floor0

        # -----------------------------------------------------------------------------------------------------------------------------
        # Initiate the spawn map and mutate the csv to fit the required array
        # -----------------------------------------------------------------------------------------------------------------------------
        spawn = pd.read_csv("spawnmap.csv", header=None)
        spawn = pd.DataFrame.to_numpy(spawn)
        spawn = np.flipud(spawn)
        self.spawnmap = spawn

        # Initialize cumulative exited count
        self.cumulative_exited = 0
        self.exited_ids = []

        # Initialize DataCollector (MESA tool for tracking metrics across steps)
        self.datacollector = DataCollector(
            model_reporters={
                "Active Agents": lambda m: len(m.agents),  # Count of agents still active
                "Exited Agents": lambda m: sum(1 for agent in m.agents if agent.found_exit == True),
                "Cumulative Exited Agents": lambda m: m.cumulative_exited,  # Cumulative exited count
                "Staff per Cell": self.count_staff_per_cell, # Counts staff in each cell
                "Mobile Agents per Cell": self.count_mobile_agents_per_cell,
                "Wheelchair Agents per Cell": self.count_chair_agents_per_cell  # Counts agents in each cell# Counts agents in each cell
            },
            agent_reporters={
                "Found Exit": lambda a: a.found_exit if isinstance(a, (MobileAgent, StaffAgent)) else None  # Reports exit status per agent
            }
        )

        self.place_agents(agent_vision)  # Place agents on the grid
        self.firstfloor_agents = []
        # self.cumulative_chair_agents_not_exited = sum(1 for obj in self.agents if isinstance(obj, WheelchairAgent)) # stores the number of WheelchairAgents yet to exit
        ## commented lines to check if staff on first floor indeed stays there until all WheelchairAgents exit
        # self.initial_first_floor_staff = 0 # stores the number of staff on the first floor after spawning
        # self.first_floor_staff_counter = 0 # stores the current number of staff on the first floor
        self.datacollector.collect(self) # Collect data at the start of the simulation

    # Function to place agents on the grid
    def place_agents(self, agent_vision):
        # If the spawn map is not used, place the specified number of each agent on the grid
        if settings['run_params']['use_map'] == False:
            for i in range(self.num_staff):
                agent = StaffAgent(self, agent_vision)
                placed = False  # Track if the agent is successfully placed
                while not placed:
                    x = self.random.randrange(self.grid.width)
                    y = self.random.randrange(self.grid.height)
                    cell_contents = self.grid.get_cell_list_contents((x, y))
                    # Only place agent if cell has no obstacles and fewer than 8 agents
                    if (x, y) in self.indoors and len(cell_contents) < settings['run_params']['tile_pop']:
                        self.grid.place_agent(agent, (x, y))  # MESA function to place agent in grid
                        placed = True  # Mark as placed

            for i in range(self.num_mobile_agents):
                agent = MobileAgent(self, agent_vision)
                placed = False  # Track if the agent is successfully placed
                while not placed:
                    x = self.random.randrange(self.grid.width)
                    y = self.random.randrange(self.grid.height)
                    cell_contents = self.grid.get_cell_list_contents((x, y))
                    # Only place agent if cell has no obstacles and fewer than 8 agents
                    if (x, y) in self.indoors and len(cell_contents) < settings['run_params']['tile_pop']:
                        self.grid.place_agent(agent, (x, y))  # MESA function to place agent in grid
                        placed = True  # Mark as placed

            for i in range(self.num_chair_agents):
                agent = WheelchairAgent(self, agent_vision)
                placed = False  # Track if the agent is successfully placed
                while not placed:
                    x = self.random.randrange(self.grid.width)
                    y = self.random.randrange(self.grid.height)
                    cell_contents = self.grid.get_cell_list_contents((x, y))
                    # Only place agent if cell has no obstacles and fewer than 8 agents
                    if (x, y) in self.indoors and len(cell_contents) < settings['run_params']['tile_pop']:
                        self.grid.place_agent(agent, (x, y))  # MESA function to place agent in grid
                        placed = True  # Mark as placed
        else:
            # If the spawn map is used, pick a random location and pick type of agent according to spawn fractions
            for i in range(self.total_agents):
                placed = False
                while not placed:
                    # Pick random location
                    x = self.random.randrange(self.grid.width)
                    y = self.random.randrange(self.grid.height)
                    cell_contents = self.grid.get_cell_list_contents((x, y))
                    # Only place agent if cell has no obstacles and fewer than 8 agents
                    if (x, y) in self.indoors and len(cell_contents) < settings['run_params']['tile_pop']:
                        # Get the type of room from the spawn map and its spawn distribution
                        index  = self.spawnmap[y, x]
                        frac_mobile = settings['spawn_rules'][str(index)]['mobile']
                        frac_wheelchair = settings['spawn_rules'][str(index)]['wheelchair']
                        frac_staff = settings['spawn_rules'][str(index)]['staff']

                        # Decide on the type of agent according to spawn fractions
                        agent_frac = np.random.random()
                        if agent_frac <= frac_mobile:
                            agent = MobileAgent(self, agent_vision)
                        elif agent_frac <= frac_mobile + frac_wheelchair:
                            agent = WheelchairAgent(self, agent_vision)
                        elif agent_frac <= frac_mobile + frac_wheelchair + frac_staff:
                            agent = StaffAgent(self, agent_vision)

                        # Place agent
                        self.grid.place_agent(agent, (x, y))  # MESA function to place agent in grid
                        placed = True  # Mark as placed


    # Function to count staff in each cell
    def count_staff_per_cell(self):
        staff_counts = {}  # Dictionary to store staff counts by cell position
        # MESA `coord_iter` function iterates over grid cells and their contents
        for cell in self.grid.coord_iter():
            cell_contents, (x, y) = cell  # Unpack cell contents and coordinates
            # Count StaffAgents in each cell
            staff_agent_count = sum(1 for obj in cell_contents if isinstance(obj, StaffAgent))
            if staff_agent_count > 0:
                staff_counts[(x, y)] = staff_agent_count
        return staff_counts

    # Function to count MobileAgents in each cell
    def count_mobile_agents_per_cell(self):
        mobile_agent_counts = {}  # Dictionary to store agent counts by cell position
        # MESA `coord_iter` function iterates over grid cells and their contents
        for cell in self.grid.coord_iter():
            cell_contents, (x, y) = cell  # Unpack cell contents and coordinates
            # Count MobileAgents in each cell
            nav_mobile_agent_count = sum(1 for obj in cell_contents if isinstance(obj, MobileAgent))
            if nav_mobile_agent_count > 0:
                mobile_agent_counts[(x, y)] = nav_mobile_agent_count
        return mobile_agent_counts

    # Function to count WheelchairAgents in each cell
    def count_chair_agents_per_cell(self):
        chair_agent_counts = {}  # Dictionary to store agent counts by cell position
        # MESA `coord_iter` function iterates over grid cells and their contents
        for cell in self.grid.coord_iter():
            cell_contents, (x, y) = cell  # Unpack cell contents and coordinates
            # Count MobileAgents in each cell
            nav_chair_agent_count = sum(1 for obj in cell_contents if isinstance(obj, WheelchairAgent))
            if nav_chair_agent_count > 0:
                chair_agent_counts[(x, y)] = nav_chair_agent_count
        return chair_agent_counts

    # Function to get the staff location
    def get_staff_locations(self):
        staff_locations_dict = {}
        for agent in self.agents:
            if isinstance(agent, StaffAgent):
                staff_locations_dict[agent.unique_id] = agent.pos
        return staff_locations_dict

    # Function to get the grid data for visualization
    # copied from the scenario with elevators
    def get_grid(self):
        grid_data = np.zeros((self.grid.height, self.grid.width))
        # Mark obstacles on the grid
        for x, y in self.obstacles:
            grid_data[y, x] = 1
        # Mark indoors walking area on the grid
        for x, y in self.indoors:
            grid_data[y, x] = 2
        # Mark exit
        for x, y in self.exit_location:
            grid_data[y, x] = 3
        # Mark stairs on the grid
        for x, y in self.stairs:
            grid_data[y, x] = 4
        # Mark lifts on the grid
        for x, y in self.lifts:
            grid_data[y, x] = 5
        # Mark StaffAgents on the grid
        for agent in self.agents:
            if isinstance(agent, StaffAgent):
                x, y = agent.pos
                grid_data[y, x] = 6
        # Mark MobileAgents on the grid
        for agent in self.agents:
            if isinstance(agent, MobileAgent):
                x, y = agent.pos
                grid_data[y, x] = 7
        # Mark WheelchairAgents on the grid
        for agent in self.agents:
            if isinstance(agent, WheelchairAgent):
                x, y = agent.pos
                grid_data[y, x] = 8

        return grid_data

    # Model step function to update the simulation
    def step(self):
        # print("step:", self.step_count)
        self.staff_locations = self.get_staff_locations() # to get the staff locations before agents' step function is executed
        # Ordered steps to make the helping work
        self.agents_by_type[StaffAgent].do("step")
        self.agents_by_type[MobileAgent].do("step")
        self.agents_by_type[WheelchairAgent].do("step")
        self.datacollector.collect(self)  # MESA DataCollector collects metrics at each step
        self.staff_locations = {} # to start the next step with an empty dictionary
        self.step_count += 1


In [ ]:
# Run the model and see the results
model = FloorPlanModel(settings['run_params']['n_staff'], settings['run_params']['n_mobile'], settings['run_params']['n_chair'], settings['run_params']['n_total'], settings['run_params']['vis_range'])
for i in range(settings['run_params']['steps']):
    model.step() # step the model by 1

# Collect the data from the model
model_data = model.datacollector.get_model_vars_dataframe()
# agent_data = model.datacollector.get_agent_vars_dataframe()

if __name__ == '__main__':
    params = {"num_staff": 10, "num_mobile_agents": 10, "num_chair_agents": 10, "tot_agents": 2000, "agent_vision": 10}
    results1 = batch_run(
        FloorPlanModel,
        parameters=params,
        iterations=10,
        max_steps=300,
        number_processes=1,
        data_collection_period=1,
        display_progress=True
    )

In [ ]:
batch_1D_results = pd.DataFrame(results1).to_csv("files/results_without_elevators.csv")

In [21]:
agent_data # print the agent data

Found Exit
Step AgentID           
0    1            False
     2            False
     3             None
     4             None
     5            False
...                 ...
10   96            None
     97           False
     98            None
     99           False
     100           None

[1094 rows x 1 columns]

In [22]:
model_data # print the model data

,Active Agents,Exited Agents,Cumulative Exited Agents,Staff per Cell,Mobile Agents per Cell,Wheelchair Agents per Cell
0,100,0,0,"{(18, 113): 1, (20, 11): 1, (23, 93): 1, (24, ...","{(3, 47): 1, (5, 58): 1, (7, 114): 1, (9, 49):...","{(6, 47): 1, (13, 114): 1, (20, 11): 1, (24, 4..."
1,100,0,0,"{(17, 113): 1, (21, 11): 1, (23, 57): 1, (24, ...","{(4, 47): 1, (4, 58): 1, (8, 114): 1, (10, 49)...","{(7, 47): 1, (15, 114): 1, (20, 10): 1, (25, 4..."
2,100,0,0,"{(17, 114): 1, (22, 11): 1, (22, 57): 1, (25, ...","{(4, 48): 1, (4, 57): 1, (9, 114): 1, (11, 49)...","{(8, 47): 1, (16, 114): 1, (21, 10): 1, (25, 4..."
3,100,0,0,"{(17, 115): 1, (21, 57): 1, (23, 11): 1, (25, ...","{(4, 49): 1, (4, 56): 1, (9, 115): 1, (12, 49)...","{(9, 47): 1, (17, 114): 1, (22, 10): 1, (25, 4..."
4,100,0,0,"{(17, 116): 1, (20, 57): 1, (24, 11): 1, (25, ...","{(4, 50): 1, (4, 55): 1, (9, 116): 1, (13, 49)...","{(10, 47): 1, (17, 115): 1, (23, 10): 1, (25, ..."
5,100,0,0,"{(17, 117): 1, (19, 57): 1, (24, 57): 1, (25, ...","{(4, 51): 1, (4, 54): 1, (9, 117): 1, (12, 55)...","{(11, 47): 1, (17, 116): 1, (24, 10): 1, (26, ..."
6,100,0,0,"{(17, 118): 1, (18, 57): 1, (23, 57): 1, (25, ...","{(3, 51): 1, (4, 53): 1, (9, 118): 1, (11, 55)...","{(12, 47): 1, (17, 117): 1, (25, 10): 1, (27, ..."
7,99,0,1,"{(17, 57): 1, (17, 119): 1, (22, 57): 1, (25, ...","{(2, 51): 1, (4, 52): 1, (9, 119): 1, (10, 55)...","{(13, 47): 1, (17, 118): 1, (25, 9): 1, (27, 4..."
8,99,0,1,"{(16, 119): 1, (17, 56): 1, (21, 57): 1, (25, ...","{(1, 51): 1, (3, 51): 1, (9, 55): 1, (10, 119)...","{(14, 47): 1, (17, 119): 1, (25, 8): 1, (27, 4..."
9,99,0,1,"{(15, 119): 1, (17, 55): 1, (20, 57): 1, (26, ...","{(0, 51): 1, (2, 51): 1, (8, 54): 1, (9, 119):...","{(15, 47): 1, (16, 119): 1, (26, 8): 1, (27, 4..."


# Visualization with User Interface
You don't have to understand every single line in the visualisation code below. Please understand the code to extend that you can introduce changes to it when needed

In [23]:
# Visualization Function
def plot_grid(model, ax):
    rc("animation", embed_limit=4096)  # Set a higher limit in MB to allow smoother animation playback
    grid_data = model.get_grid()  # Retrieve the current state of the grid from the model
    ax.clear()  # Clear any previous plots on the axes to prevent overlap in visualizations

    # Define color mappings:
    # 0 (outdoors) -> green, 1 (obstacle) -> black, 2 (indoors) -> white,
    # 3 (door) -> white, 4 (exit) -> purple/mediumorchid, 5 (yellow) -> stairs, 6 (lifts) -> orange, 7 (StaffAgent) -> red, 8 (MobileAgent) -> blue, 9 (WheelchairAgent) -> pink
    cmap = mcolors.ListedColormap(['green', 'black', 'white', 'mediumorchid', 'yellow', 'orange', 'red', 'blue', 'lime'])
    bounds = [0, 1, 2, 3, 4, 5, 6, 7, 8]  # Boundaries to separate each category
    norm = mcolors.BoundaryNorm(bounds, cmap.N, extend='max')  # Normalizes values to assign colors to each category

    # Setting 'origin' to 'lower' places the (0,0) coordinate at the bottom-left.
    ax.imshow(grid_data, cmap=cmap, norm=norm, origin='lower')

    # Customize grid display: set grid to start from -0.5 with labels at intervals of 1
    # Set minor ticks for cell boundaries, with thicker and darker lines
    ax.set_xticks(np.arange(-0.5, model.grid.width, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, model.grid.height, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=0.3)  # Thicker, darker lines for cell boundaries

    # Set major ticks for labels at intervals of 5, with lighter and thinner lines
    ax.set_xticks(np.arange(0, model.grid.width, 5), minor=False)
    ax.set_yticks(np.arange(0, model.grid.height, 5), minor=False)
    ax.grid(which='major', color='lightgray', linestyle='-', linewidth=0)  # invisible line

    # Set the title to show the current step number in the model
    ax.set_title(f"Step {model.steps}", fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=7)  # Control tick label size

    # Draw movement arrows for agents to show the direction they are traveling
    for agent in model.agents:
        # Draw arrow if the agent has moved and is a StaffAgent
        if isinstance(agent, StaffAgent) and agent.previous_pos:
            start_x, start_y = agent.previous_pos  # Previous position of the agent
            end_x, end_y = agent.pos  # Current position of the agent

            # Draw an arrow from the previous position to the current position
            ax.arrow(
                start_x, start_y,
                end_x - start_x, end_y - start_y,
                head_width=0.3, head_length=0.3, fc='yellow', ec='yellow'  # Yellow arrow for movement direction
            )
        # Draw arrow if the agent has moved and is a MobileAgent
        if isinstance(agent, MobileAgent) and agent.previous_pos:
            start_x, start_y = agent.previous_pos  # Previous position of the agent
            end_x, end_y = agent.pos  # Current position of the agent

            # Draw an arrow from the previous position to the current position
            ax.arrow(
                start_x, start_y,
                end_x - start_x, end_y - start_y,
                head_width=0.3, head_length=0.3, fc='yellow', ec='yellow'  # Yellow arrow for movement direction
            )

        # Draw arrow if the agent has moved and is a WheelchairAgent
        if isinstance(agent, WheelchairAgent) and agent.previous_pos:
            start_x, start_y = agent.previous_pos  # Previous position of the agent
            end_x, end_y = agent.pos  # Current position of the agent

            # Draw an arrow from the previous position to the current position
            ax.arrow(
                start_x, start_y,
                end_x - start_x, end_y - start_y,
                head_width=0.3, head_length=0.3, fc='yellow', ec='yellow'  # Yellow arrow for movement direction
            )
# Animation Update Function
def update(frame, model, ax):
    # For every frame, update the model state if it's not the first frame
    if frame > 0:
        model.step()  # Run one step of the model simulation
        print(f"Running Simulation Step: {model.steps}")
    plot_grid(model, ax)  # Redraw the grid with updated agent positions

# Run the Animation with a larger figure size
def run_animation(model, steps):
    fig, ax = plt.subplots(figsize=(settings['run_params']['figsize'], settings['run_params']['figsize']))  # Create a 10x10 figure for the plot
    fig.tight_layout()
    plot_grid(model, ax)  # Plot the initial grid state
    # Create an animation that updates the grid for each step
    anim = FuncAnimation(fig, update, frames=steps+1, fargs=(model, ax), repeat=False)
    plt.close(fig)  # Close the figure after animation creation to avoid duplicate displays
    return anim

# Your imports and function definitions remain the same, up until where you initialize and run the model
import ipywidgets as widgets  # For interactive widgets (slider, button)
from IPython.display import display, HTML  # To display widgets and HTML animations
import time  # For tracking elapsed time

# Slider to choose the number of StaffAgents
if settings['run_params']['use_map'] == False:
    staff_slider = widgets.IntSlider(
        value=settings['run_params']['n_staff'],      # Default starting number of agents
        min=0,        # Minimum number of agents allowed
        max=200,       # Maximum number of agents allowed
        step=5,       # Step size for slider increments
        description='Num Staff:',  # Label for the slider
        continuous_update=False     # Only update value when slider is released
    )

    # Slider to choose the number of MobileAgents
    mobile_slider = widgets.IntSlider(
        value=settings['run_params']['n_mobile'],      # Default starting number of agents
        min=0,        # Minimum number of agents allowed
        max=500,       # Maximum number of agents allowed
        step=10,       # Step size for slider increments
        description='Num Mobile:',  # Label for the slider
        continuous_update=False     # Only update value when slider is released
    )

    # Slider to choose the number of WheelchairAgents
    chair_slider = widgets.IntSlider(
        value=settings['run_params']['n_chair'],      # Default starting number of agents
        min=0,        # Minimum number of agents allowed
        max=200,       # Maximum number of agents allowed
        step=10,       # Step size for slider increments
        description='Num Chair:',  # Label for the slider
        continuous_update=False     # Only update value when slider is released
    )

else:
    agent_slider = widgets.IntSlider(
        value=settings['run_params']['n_total'],      # Default starting number of agents
        min=10,        # Minimum number of agents allowed
        max=2000,       # Maximum number of agents allowed
        step=10,       # Step size for slider increments
        description='Num Agents:',  # Label for the slider
        continuous_update=False     # Only update value when slider is released
    )

# Slider to choose the number of time steps (frames) for the animation
time_step_slider = widgets.IntSlider(
    value=settings['run_params']['steps'],      # Default starting number of steps
    min=1,         # Minimum time steps allowed
    max=600,       # Maximum time steps allowed
    step=1,        # Step size for slider increments
    description='Time Steps:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Slider to choose the vision range (vision radius) of agents
vision_slider = widgets.IntSlider(
    value=settings['run_params']['vis_range'],      # Default starting vision range
    min=1,        # Minimum vision range
    max=20,       # Maximum vision range
    step=1,       # Step size for slider increments
    description='Vision:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Button to start the simulation with current slider settings
run_button = widgets.Button(description="Run Simulation")

# Output widget for displaying the animation and elapsed time
output_widget = widgets.Output()

# Label to show elapsed time during the simulation
elapsed_time_label = widgets.Label(value="Elapsed time: 0.0 seconds")

# Flag variable to control timing function
stop_timer = False

# Display the interface with sliders, button, label, and output area
if settings['run_params']['use_map'] == False:
    display(staff_slider, mobile_slider, chair_slider, time_step_slider, vision_slider, run_button, elapsed_time_label, output_widget)
else:
    display(agent_slider, time_step_slider, vision_slider, run_button, elapsed_time_label, output_widget)

# Define the function to initialize and run the model
def run_model(change):
    global stop_timer, model_data, agent_data  # Allow variables to be accessed globally
    with output_widget:  # Use output widget to display the animation
        output_widget.clear_output()  # Clear any previous output
        time_steps = time_step_slider.value  # Get the number of steps from the slider
        agent_vision = vision_slider.value  # Get the vision range from the slider
        if settings['run_params']['use_map'] == False:
            num_staff = staff_slider.value
            num_mobile = mobile_slider.value
            num_chair = chair_slider.value
            model = FloorPlanModel(num_staff=num_staff, num_mobile_agents=num_mobile, num_chair_agents=num_chair,  tot_agents=settings['run_params']['n_total'] , agent_vision=agent_vision)  # Initialize the model
        else:
            num_total = agent_slider.value
            model = FloorPlanModel(num_staff=settings['run_params']['n_staff'], num_mobile_agents=settings['run_params']['n_mobile'],num_chair_agents=settings['run_params']['n_chair'],  tot_agents=num_total, agent_vision=agent_vision)


        # Reset timer flag and start timing for the animation
        stop_timer = False
        start_time = time.time()

        def update_time_label():
            while not stop_timer:  # Keep updating time label until timer stops
                elapsed_time = time.time() - start_time
                elapsed_time_label.value = f"Elapsed time: {elapsed_time:.1f} seconds"
                time.sleep(0.1)  # Update every 0.1 seconds for real-time effect

        # Start elapsed time tracking in a separate thread
        import threading
        timer_thread = threading.Thread(target=update_time_label, daemon=True)
        timer_thread.start()

        # Run the animation with selected number of steps
        anim = run_animation(model, steps=time_steps)

        # Display the animation output in HTML format
        output = HTML(anim.to_jshtml())
        display(output)

        # Stop the timer after the simulation completes
        stop_timer = True
        elapsed_time = time.time() - start_time
        elapsed_time_label.value = f"Total elapsed time: {elapsed_time:.1f} seconds"

        # Retrieve and display model and agent data after simulation completes
        model_data = model.datacollector.get_model_vars_dataframe()  # Data for the model over time
        agent_data = model.datacollector.get_agent_vars_dataframe()  # Data for each agent over time
    return model, model_data, agent_data  # Return model and data for further inspection

# Attach the run_model function to the run button click event
run_button.on_click(run_model)

IntSlider(value=100, continuous_update=False, description='Num Agents:', max=2000, min=10, step=10)

IntSlider(value=10, continuous_update=False, description='Time Steps:', max=600, min=1)

IntSlider(value=10, continuous_update=False, description='Vision:', max=20, min=1)

Button(description='Run Simulation', style=ButtonStyle())

Label(value='Elapsed time: 0.0 seconds')

Output()

In [ ]:
print("\nModel Data:")
model_data

In [ ]:
print("\nAgent Data:")
agent_data.head(40)


In [ ]:
#  Uncomment the code below to display the 'Agents per Cell' data for each step
